In [ ]:
!pip install pyswarm

In [ ]:
# from sklearn.metrics import mean_squared_error, mean_absolute_error
# from keras.layers import Input, Dense, GaussianNoise, Dropout
# from tensorflow.keras.callbacks import EarlyStopping
# from sklearn.model_selection import train_test_split
# from tensorflow.keras import backend as K
# from keras import regularizers
# from google.colab import drive
# from keras.models import Model
# from datetime import datetime
# from tensorflow import keras
# import tensorflow as tf
# from pyswarm import pso
# from math import sqrt
# import pandas as pd
# import numpy as np

In [ ]:
from tensorflow.keras.layers import Input, Dense, GaussianNoise, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from tensorflow.keras.layers import Flatten
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model
from google.colab import drive
from datetime import datetime
from tensorflow import keras
from pyswarm import pso
import pandas as pd
import numpy as np
import time

In [ ]:
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Volatility/New_data_2000_2024/Not_normal_wd/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def set_index(df):
  df.index = pd.to_datetime(df['Date'])
  df.drop(columns=['Date'], inplace=True)

In [ ]:
filename = 'PV_data.csv'
PV_data = pd.read_csv(path + filename)

In [ ]:
set_index(PV_data)

In [ ]:
tm_data = PV_data

In [ ]:
PV_data = PV_data.drop(columns=['PV'])

In [ ]:
PV_data['PV'] = tm_data['PV']

In [ ]:
PV_data

,"('GARCH', 'normal', 0)","('GARCH', 'gaussian', 0)","('GARCH', 'ged', 0)","('FIGARCH', 'normal', 0)","('FIGARCH', 'normal', 1)","('FIGARCH', 'gaussian', 0)","('FIGARCH', 'gaussian', 1)","('FIGARCH', 'ged', 0)","('FIGARCH', 'ged', 1)",gdp_growth,...,usd_eur,usd_jpy,usd_gbp,usd_cny,usd_cad,usd_mxn,gt_data,log_returns,"('APARCH', 'studentst', 1)",PV
Date,,,,,,,,,,,,,,,,,,,,,
2000-02-02,0.039601,0.039601,0.040044,0.038388,0.038388,0.038388,0.038388,0.038865,0.038865,28984.929215,...,2.781970,307.252039,4.529620,23.415205,4.090860,26.750733,209.303607,-0.005604,0.037128,0.035334
2000-02-03,0.037979,0.037979,0.041000,0.038633,0.038633,0.038633,0.038633,0.040342,0.040342,28984.929215,...,2.799436,311.491144,4.529266,23.412659,4.119569,26.561052,209.303607,-0.008693,0.038336,0.036011
2000-02-04,0.036791,0.036791,0.038214,0.036990,0.036990,0.036990,0.036990,0.039265,0.039265,28984.929215,...,2.728548,306.262089,4.474254,23.415452,4.106629,26.433950,209.303607,0.000572,0.049778,0.034207
2000-02-07,0.046593,0.046593,0.047968,0.046474,0.046474,0.046474,0.046474,0.048168,0.048168,28984.929215,...,2.733357,299.491542,4.460147,23.414391,4.140641,26.339020,209.303607,0.027307,0.065732,0.038502
2000-02-08,0.049131,0.049131,0.049560,0.046711,0.046711,0.046711,0.046711,0.045562,0.045562,28984.929215,...,2.730104,301.761354,4.470789,23.415982,4.142338,26.135551,209.303607,0.010478,0.043373,0.041434
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-26,0.000120,0.000120,0.000456,0.000121,0.000121,0.000121,0.000121,0.000423,0.000423,0.000000,...,-0.002546,0.509117,-0.003041,0.005162,0.001980,-0.105925,0.404061,0.008246,-0.004076,0.000011
2024-08-27,-0.003889,-0.003889,-0.003539,-0.004368,-0.004368,-0.004368,-0.004368,-0.001972,-0.001972,0.000000,...,0.002051,-0.551543,-0.001202,-0.006788,-0.001556,-0.173948,0.404061,0.009336,-0.005604,0.000029
2024-08-28,0.000280,0.000280,0.000449,0.000483,0.000483,0.000483,0.000483,0.000381,0.000381,0.000000,...,0.000849,0.247487,0.000849,0.002263,0.002051,-0.219910,0.707107,0.010307,-0.001642,0.000023


# Split train, validation and test data

In [ ]:
# def split_data(data, target, train_end =datetime(2020, 6, 30), test_start=datetime(2020, 10, 1), test_size=0.1):
#   test_data = data.loc[test_start:]

#   train_val = data.loc[:train_end]
#   train_data, val_data = train_test_split(train_val, test_size=test_size, shuffle=False)

#   X_train = train_data.drop(columns=[target])
#   y_train = train_data[target]

#   X_val = val_data.drop(columns=[target])
#   y_val = val_data[target]

#   X_test = test_data.drop(columns=[target])
#   y_test = test_data[target]

#   return np.array(X_train), np.array(y_train), np.array(X_val), np.array(y_val), np.array(X_test), np.array(y_test)

In [ ]:
def split_data(data, target, train_end =datetime(2022, 5, 30), test_start=datetime(2022, 5, 31), test_size=0.1):
  test_data = data.loc[test_start:]

  train_val = data.loc[:train_end]
  train_data, val_data = train_test_split(train_val, test_size=test_size, shuffle=False)

  return train_data, val_data, test_data

# Rolling Mechanism

In [ ]:
# Define the rolling window function
def rolling_window(df, in_sample_window_size, out_of_sample_size):
  X, y = [], []
  for i in range(in_sample_window_size, len(df) - out_of_sample_size):
    X.append(df.iloc[i - in_sample_window_size:i, :-1].values)  # All features except the target column
    y.append(df.iloc[i:i + out_of_sample_size, -1].values)  # Target column

  return np.array(X), np.array(y)

# Model

## PV

In [ ]:
# PV_X_train, PV_y_train, PV_X_val, PV_y_val, PV_X_test, PV_y_test = split_data(PV_data, 'PV')

In [ ]:
daily_train, daily_val, daily_test = split_data(PV_data, 'PV')

In [ ]:
in_sam_win_sz = 25
out_sam_win_sz = 5

In [ ]:
PV_X_train, PV_y_train = rolling_window(daily_train, in_sam_win_sz, out_sam_win_sz)
PV_X_val, PV_y_val = rolling_window(daily_val, in_sam_win_sz, out_sam_win_sz)
PV_X_test, PV_y_test = rolling_window(daily_test, in_sam_win_sz, out_sam_win_sz)

In [ ]:
PV_X_train.shape

(5025, 25, 31)

In [ ]:
PV_y_train.shape

(5025, 5)

In [ ]:
# PV_y_train = np.expand_dims(PV_y_train, axis=-1)
# PV_y_val = np.expand_dims(PV_y_val, axis=-1)
# PV_y_test = np.expand_dims(PV_y_test, axis=-1)

In [ ]:
# qloss function to calculate loss
def qloss(y_true, y_pred, n_q):
  q = np.array(range(1, n_q + 1))
  left = (q / (n_q + 1) - 1) * (y_true - y_pred)
  right = q / (n_q + 1) * (y_true - y_pred)

  return keras.backend.mean(keras.backend.maximum(left, right))

In [ ]:
# Function to create the model
def get_model(input_dim, num_units, act, dp, gauss_std, num_hidden_layers, num_quantiles):
    input = Input((input_dim[1], input_dim[2], ), name='input')

    x = input

    for i in range(num_hidden_layers):
        x = Dense(num_units[i], use_bias=True, kernel_initializer='he_normal', bias_initializer='he_normal',
                  kernel_regularizer=regularizers.l2(0.001), activation=act[i])(x)
        x = Dropout(dp[i])(x)
        x = GaussianNoise(gauss_std[i])(x)

    # Flatten the 3D input to 2D to remove the time dimension
    x = Flatten()(x)

    x = Dense(out_sam_win_sz, activation=None, use_bias=True, kernel_initializer='he_normal', bias_initializer='he_normal')(x)

    model = Model(input, x)

    return model

In [ ]:
# Objective function for PSO
def objective_function(params):
    num_hidden_layers = int(params[0])
    # num_quantiles = int(params[1])
    num_quantiles = out_sam_win_sz

    # Adjust lists to match the number of hidden layers
    num_units = [int(params[1])] * num_hidden_layers
    act = ['relu'] * num_hidden_layers
    dropout = [params[2]] * num_hidden_layers
    gauss_std = [params[3]] * num_hidden_layers

    # Create the model
    model = get_model(input_dim=input_dim, num_units=num_units, act=act, dp=dropout, gauss_std=gauss_std, num_hidden_layers=num_hidden_layers, num_quantiles=num_quantiles)

    # Compile the model
    model.compile(loss=lambda y_t, y_p: qloss(y_t, y_p, n_q=num_quantiles), optimizer='adam')

    # Train the model
    early_stopping = EarlyStopping(monitor='val_loss', patience=5)
    model.fit(PV_X_train, PV_y_train, epochs=80, validation_data=(PV_X_val, PV_y_val), batch_size=64, callbacks=[early_stopping], verbose=0)

    # Predictions
    predictions = model.predict(PV_X_val)

    # Calculate mean squared error (MSE) as the objective function
    mse = mean_squared_error(PV_y_val, predictions)

    return mse

In [ ]:
# # Parameter bounds for PSO
# lb = [1, 10, 50, 50, 0.0, 0.0, 0.0, 0.0]  # Lower bounds [num_hidden_layers, num_quantiles, ...]
# ub = [5, 99, 300, 300, 0.5, 0.5, 1.0, 1.0]  # Upper bounds [num_hidden_layers, num_quantiles, ...]

# input_dim = 25

# swarmsize = 50
# maxiter = 100

In [ ]:
# Parameter bounds for PSO
lb = [1, 30, 0.1, 0.0]  # Lower bounds [num_hidden_layers, num_quantiles, ...]
ub = [3, 80, 0.3, 0.5]  # Upper bounds [num_hidden_layers, num_quantiles, ...]

input_dim = PV_X_train.shape

swarmsize = 20
maxiter = 50

In [ ]:
start_time = time.time()

# Run PSO to find the best parameters
best_params, _ = pso(objective_function, lb, ub, swarmsize=swarmsize, maxiter=maxiter)

end_time = time.time()
execution_time = end_time - start_time

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
17/17 ━━━━━━━━━━━━━━━━━━━

In [ ]:
execution_time

16356.617552280426

In [ ]:
best_params

array([ 3.        , 64.2677709 ,  0.10702979,  0.        ])

In [ ]:
# best_params =  [1.67780478, 30, 0.10639287, 0]

In [ ]:
# best_params = [2.44767625, 50.70353665, 0.26700986, 0.42920676]
# best_params = [1.71454653, 30.57286514, 0.21601151, 0.28231749]
# best_params = [1.05013429, 79.89863874,  0.13223594,  0.]

In [ ]:
num_hidden_layers = int(best_params[0])
# num_quantiles = int(best_params[1])
num_quantiles = out_sam_win_sz

# Adjust lists to match the number of hidden layers
num_units = [int(best_params[1])] * num_hidden_layers
act = ['relu'] * num_hidden_layers
dropout = [best_params[2]] * num_hidden_layers
gauss_std = [best_params[3]] * num_hidden_layers

# Create the model
QRNN_model = get_model(input_dim=input_dim, num_units=num_units, act=act, dp=dropout, gauss_std=gauss_std, num_hidden_layers=num_hidden_layers, num_quantiles=num_quantiles)

# Compile the model
QRNN_model.compile(loss=lambda y_t, y_p: qloss(y_t, y_p, n_q=num_quantiles), optimizer='adam')

# Train the model
early_stopping = EarlyStopping(monitor='val_loss', patience=5)
QRNN_model.fit(PV_X_train, PV_y_train, epochs=80, validation_data=(PV_X_val, PV_y_val), batch_size=64, callbacks=[early_stopping], verbose=0)

In [44]:
# Predictions
PV_test_predictions = QRNN_model.predict(PV_X_test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step


In [45]:
# # Train and predict with optimal parameters
# PV_test_predictions = esn_model(optimal_params, PV_X_train, PV_y_train, PV_X_test)

In [46]:
def calculate_mape(actual, predicted):
  actual, predicted = np.array(actual), np.array(predicted)
  return np.mean(np.abs((actual - predicted) / actual)) * 100

In [47]:
mse = mean_squared_error(PV_y_test, PV_test_predictions)
print(f'MSE: {mse}')

rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

mae = mean_absolute_error(PV_y_test, PV_test_predictions)
print(f'MAE: {mae}')

mape = calculate_mape(PV_y_test, PV_test_predictions)
print(f'MAPE: {mape}%')

MSE: 7.584052384856711e-08
RMSE: 0.0002753915827482153
MAE: 0.00021000142173987585
MAPE: 706.5609593007601%


In [48]:
# path_1 = '/content/drive/MyDrive/Volatility/New_data_2000_2024/New_predictions/PV/'

# df_pred = pd.DataFrame(PV_test_predictions[:,0], columns=['qrnn_pred'])
# filename = 'QRNN_pred.csv'
# df_pred.to_csv(path_1 + filename)